In [2]:
from sklearn import metrics  
import numpy as np  
import pandas as pd
import datetime
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
#########SKLearn 集成的算法###############
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree  
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB 
#########SKLearn 集成的算法###############
from sklearn.model_selection import train_test_split # 切分数据
from sklearn.metrics import mean_squared_error #评价指标

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings
warnings.filterwarnings("ignore")
 
%matplotlib inline

In [3]:
############全局参数#################################
id_col_names=['user_id','coupon_id','date_received']
target_col_name='label'
id_target_cols=['user_id','coupon_id','date_received','label']
myeval='roc_auc'
cvscore=0
############目录定义#################################
datapath = 'data/' 
featurepath = 'feature/' 
resultpath = 'result/'
tmppath = 'tmp/'
scorepath = 'score/'
###########工具函数#############################################

In [4]:
###########工具函数#############################################
#返回ID列
def get_id_df(df):
    return df[id_col_names]

#返回Target列
def get_target_df(df):
    return df[target_col_name]

#返回特征列
def get_predictors_df(df):
    predictors = [f for f in df.columns if f not in id_target_cols]
    return df[predictors]

#按特征名读取训练集
def read_featurefile_train(featurename): 
    df=pd.read_csv(featurepath+'train_'+featurename+'.csv', sep=',' , encoding = "utf-8")
    df.fillna(0,inplace=True)
    return df

#按特征名读取测试集
def read_featurefile_test(featurename): 
    df=pd.read_csv(featurepath+'test_'+featurename+'.csv', sep=',' , encoding = "utf-8")
    df.fillna(0,inplace=True)
    return df

#按特征名读取数据
def read_data(featurename): 
    traindf = read_featurefile_train(featurename)
    testdf = read_featurefile_test(featurename)
    return traindf,testdf  

In [5]:
# 将特征归一化
def standize_df(train_data,test_data):
    from sklearn import preprocessing 
    
    features_columns = [f for f in test_data.columns if f not in id_target_cols]
    min_max_scaler = preprocessing.MinMaxScaler()
    min_max_scaler = min_max_scaler.fit(train_data[features_columns])
    
    train_data_scaler = min_max_scaler.transform(train_data[features_columns])
    test_data_scaler = min_max_scaler.transform(test_data[features_columns])
    
    train_data_scaler = pd.DataFrame(train_data_scaler)
    train_data_scaler.columns = features_columns
    
    test_data_scaler = pd.DataFrame(test_data_scaler)
    test_data_scaler.columns = features_columns
    
    train_data_scaler['label'] = train_data['label']
    train_data_scaler[id_col_names] = train_data[id_col_names]
    test_data_scaler[id_col_names] = test_data[id_col_names]
    return train_data_scaler,test_data_scaler

In [148]:
####################部分整合在sklearn的分类算法###############
from sklearn.neural_network import MLPClassifier
def get_sklearn_model(model_name):
    #朴素贝叶斯
    if model_name=='NB':
        return MultinomialNB(alpha=0.01)
    #逻辑回归
    elif model_name=='LR':
        return LogisticRegression(penalty='l2') 
    # KNN  
    elif model_name=='KNN':
        return KNeighborsClassifier()  
    #随机森林
    elif model_name=='RF':
        return RandomForestClassifier()  
    #决策树
    elif model_name=='DT':
        return tree.DecisionTreeClassifier()  
    #向量机
    elif model_name=='SVC':
        return SVC(kernel='rbf')
    #GBDT
    elif model_name=='GBDT':
        return GradientBoostingClassifier()
    #XGBoost
    elif model_name=='XGB':
        return XGBClassifier()
    #lightGBM
    elif model_name=='LGB':
        return LGBMClassifier()
    elif model_name=="MLP":
        model=MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=(50,50,50), learning_rate='constant',
        learning_rate_init=0.01, max_iter=200, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
        solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
        warm_start=False)
        return model
    else:
        print("wrong model name!")


In [7]:
#画学习曲线
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=[0.01,0.02,0.05,0.1,0.2,0.3]):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv,scoring=myeval, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
 
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
 
    plt.legend(loc="best")
    return plt

#画算法的学习曲线,为加快画图速度，最多选20%数据
def plot_curve_single(traindf,classifier,cvnum,train_sizes=[0.01,0.02,0.05,0.1,0.2,0.3]): 
    X=get_predictors_df(traindf)
    y=get_target_df(traindf)
    estimator = get_sklearn_model(classifier)    #建模
    title = "learning curve of "+classifier+", cv:"+str(cvnum)
    plot_learning_curve(estimator, title, X, y, ylim=(0, 1.01), cv=cvnum, train_sizes=train_sizes)

In [87]:
#性能评价函数
#本赛题目标是预测投放的优惠券是否核销。
#针对此任务及一些相关背景知识，使用优惠券核销预测的平均AUC（ROC曲线下面积）作为评价标准。 
#即对每个优惠券coupon_id单独计算核销预测的AUC值，再对所有优惠券的AUC值求平均作为最终的评价标准。
# coupon平均auc计算
# label就是优惠券是否核销，每个人领取的优惠券是唯一的。

def myauc(test):
    testgroup = test.groupby(['coupon_id'])
    aucs = []
    for i in testgroup:
        coupon_df = i[1]
        #测算AUC必须大于1个类别
        if len(coupon_df['label'].unique()) < 2:
            continue
        auc = metrics.roc_auc_score(coupon_df['label'], coupon_df['pred'])
        aucs.append(auc)
    return np.average(aucs)




def test_model_split(traindf,classifier):
    
    target = get_target_df(traindf).copy()
    
    train_all,test_all,train_target,test_target=train_test_split(traindf,target,test_size=0.1,random_state=0)
    
    train_data = get_predictors_df(train_all).copy()
    test_data = get_predictors_df(test_all).copy()
    
    clf = get_sklearn_model(classifier)



    result=clf.predict_proba(test_data)[:,1]
    
    test=test_all.copy()
    test['pred']=result
    
    score = metrics.roc_auc_score(test_target, result)
    print(classifier+"总体AUC:", score)
    score_coupon = myauc(test)
    print(classifier+" Coupon AUC:", score_coupon)



In [66]:
def test_model_stack(traindf,classifier_list):
    train=traindf[traindf.date_received<20160515].copy()
    test=traindf[traindf.date_received>=20160515].copy()

    train_data = get_predictors_df(train).copy()
    train_target = get_target_df(train).copy()
    test_data = get_predictors_df(test).copy()
    test_target = get_target_df(test).copy()

    flag=False
    for classifier in classifier_list:
        clf = get_sklearn_model(classifier)
        clf.fit(train_data, train_target)
        if flag==False:
            result=clf.predict_proba(test_data)[:,1]*0.3
            flag=True
        else:
            result+=clf.predict_proba(test_data)[:,1]/len(classifier_list)*0.7

    test['pred']=result
    #score = metrics.roc_auc_score(test_target, result)
    #print(" 总体 AUC:", score)
    score_coupon = myauc(test)
    print(" Coupon AUC:", score_coupon)

In [9]:
#所有的特征都是上一节生成的
train_f1,test_f1=read_data('f1')
#因为要使用KNN等进行测试，所以需要归一化
train_f1,test_f1=standize_df(train_f1,test_f1)


In [149]:
#逻辑回归
test_model(train_f3,'MLP')

MLP 总体 AUC: 0.8078207435075603
MLP Coupon AUC: 0.7240847430212439


In [68]:
test_model(train_f2,'GBDT')

GBDT 总体 AUC: 0.7828443454607813
GBDT Coupon AUC: 0.6035247998415417


In [12]:
#朴素贝叶斯
test_model(train_f1,'NB')

NB 总体 AUC: 0.6670944091690476
NB Coupon AUC: 0.5381247642711647


In [88]:
# 决策树
test_model(train_f1,'DT')

0
DT 总体 AUC: 0.6481609737147687
DT Coupon AUC: 0.5302514392404754


In [14]:
# 随机森林
test_model(train_f1,'RF')

RF 总体 AUC: 0.6480565011007058
RF Coupon AUC: 0.529662321473247


In [96]:
#按照日期分割
def test_model(traindf,classifier,pars=None):
    train=traindf[traindf.date_received<20160515].copy()
    test=traindf[traindf.date_received>=20160515].copy()

    train_data = get_predictors_df(train).copy()
    train_target = get_target_df(train).copy()
    test_data = get_predictors_df(test).copy()
    test_target = get_target_df(test).copy()
    
    clf = get_sklearn_model(classifier)
    if pars!=None:
        clf.set_params(**pars)

    clf.fit(train_data, train_target)
    result=clf.predict_proba(test_data)[:,1]
    test['pred']=result
    score = metrics.roc_auc_score(test_target, result)
    print(classifier+" 总体 AUC:", score)
    score_coupon = myauc(test)
    print(classifier+" Coupon AUC:", score_coupon)

In [74]:
params = {
    'booster': 'gbtree',
    'objective': 'binary_logloss',
    'num_class': 2,
    'num_leaves': 31,
    'subsample': 0.8,
    'bagging_freq': 1,
    'feature_fraction ': 0.8,
    'slient': 1,
    'learning_rate ': 0.01,
    'seed': 0
}

In [ ]:
params = {
    'objective': 'binary',
    'learning_rate': 0.075,
    'seed': 1024,
    'n_estimators':200,
    'max_depth':5,
    'subsample':0.8
}

In [152]:
params = {
    'objective': 'binary',
    'learning_rate': 0.075,
    'seed': 1024,
    'n_estimators':200,
    'max_depth':5,
    'subsample':0.8
}
# LightGBM
test_model(train_f3,'LGB',params)

LGB 总体 AUC: 0.823425746865561
LGB Coupon AUC: 0.7401137523907148


LGB 总体 AUC: 0.829832103489052
LGB Coupon AUC: 0.741161595882742


In [16]:
#XGBoost
test_model(train_f1,'XGB')

XGB 总体 AUC: 0.6537138075207527
XGB Coupon AUC: 0.531849037296699


In [17]:
#所有的特征都是上一节生成的
train_f2,test_f2=read_data('sf2')
#因为要使用KNN等进行测试，所以需要归一化
train_f2,test_f2=standize_df(train_f2,test_f2)

#所有的特征都是上一节生成的
train_f3,test_f3=read_data('sf3')
#因为要使用KNN等进行测试，所以需要归一化
train_f3,test_f3=standize_df(train_f3,test_f3)

In [65]:
test_model_stack(train_f3,['LR',"XGB"])

 总体 AUC: 0.8137455779006189
 Coupon AUC: 0.7354165749876264


In [18]:
print('特征f1逻辑回归成绩')
test_model(train_f1,'LR')
print('特征sf2逻辑回归成绩')
test_model(train_f2,'LR')
print('特征sf3逻辑回归成绩')
test_model(train_f3,'LR')

特征f1逻辑回归成绩
LR 总体 AUC: 0.6653536857862119
LR Coupon AUC: 0.5381247642711647
特征sf2逻辑回归成绩
LR 总体 AUC: 0.7603100610977812
LR Coupon AUC: 0.6030196306548136
特征sf3逻辑回归成绩
LR 总体 AUC: 0.8100282897146497
LR Coupon AUC: 0.7290103338100681


In [19]:
print('特征f1朴素贝叶斯成绩')
test_model(train_f1,'NB')
print('特征sf2朴素贝叶斯成绩')
test_model(train_f2,'NB')
print('特征sf3朴素贝叶斯成绩')
test_model(train_f3,'NB')

特征f1朴素贝叶斯成绩
NB 总体 AUC: 0.6670944091690476
NB Coupon AUC: 0.5381247642711647
特征sf2朴素贝叶斯成绩
NB 总体 AUC: 0.7344423161179449
NB Coupon AUC: 0.6063348615415269
特征sf3朴素贝叶斯成绩
NB 总体 AUC: 0.7812536833968516
NB Coupon AUC: 0.7119101385059943


In [20]:
print('特征f1决策树成绩')
test_model(train_f1,'DT')
print('特征sf2决策树成绩')
test_model(train_f2,'DT')
print('特征sf3决策树成绩')
test_model(train_f3,'DT')

特征f1决策树成绩
DT 总体 AUC: 0.6481607651168957
DT Coupon AUC: 0.5302514392404754
特征sf2决策树成绩
DT 总体 AUC: 0.5179836460117524
DT Coupon AUC: 0.5171234622739289
特征sf3决策树成绩
DT 总体 AUC: 0.5721348787453637
DT Coupon AUC: 0.5726505408639466


In [21]:
print('特征f1随机森林成绩')
test_model(train_f1,'RF')
print('特征sf2随机森林成绩')
test_model(train_f2,'RF')
print('特征sf3随机森林成绩')
test_model(train_f3,'RF')

特征f1随机森林成绩
RF 总体 AUC: 0.6473093832452281
RF Coupon AUC: 0.530107541180319
特征sf2随机森林成绩
RF 总体 AUC: 0.6904256599972033
RF Coupon AUC: 0.56271683410204
特征sf3随机森林成绩
RF 总体 AUC: 0.7754596079984429
RF Coupon AUC: 0.7080447915343251


In [22]:
print('特征f1 LightGBM成绩')
test_model(train_f1,'LGB')
print('特征sf2  LightGBM成绩')
test_model(train_f2,'LGB')
print('特征sf3  LightGBM成绩')
test_model(train_f3,'LGB')

特征f1 LightGBM成绩
LGB 总体 AUC: 0.6534077718076291
LGB Coupon AUC: 0.5327884933599097
特征sf2  LightGBM成绩
LGB 总体 AUC: 0.7741029767659218
LGB Coupon AUC: 0.5984526360012944
特征sf3  LightGBM成绩
LGB 总体 AUC: 0.8284887876924993
LGB Coupon AUC: 0.7374344321731373


In [23]:
print('特征f1 XGBoost成绩')
test_model(train_f1,'XGB')
print('特征sf2  XGBoost成绩')
test_model(train_f2,'XGB')
print('特征sf3  XGBoost成绩')
test_model(train_f3,'XGB')

特征f1 XGBoost成绩
XGB 总体 AUC: 0.6537138075207527
XGB Coupon AUC: 0.531849037296699
特征sf2  XGBoost成绩
XGB 总体 AUC: 0.7274131700360452
XGB Coupon AUC: 0.5886899184606202
特征sf3  XGBoost成绩
XGB 总体 AUC: 0.7973908059790942
XGB Coupon AUC: 0.7290342232690418


In [155]:
#预测函数
def classifier_df_simple(train_feat, test_feat,classifier,params):
    model = get_sklearn_model(classifier)
    model.set_params(**params) 
    model.fit(get_predictors_df(train_feat), get_target_df(train_feat))    
    predicted = pd.DataFrame(model.predict_proba(get_predictors_df(test_feat))[:,1])
    return predicted
#输出结果函数
def output_predicted(predicted,resultfile,test_feat):
    predicted=round(predicted,3)
    resultdf=get_id_df(test_feat).copy()
    resultdf['Probability']=predicted
    return resultdf

In [42]:
def classifier_df_stack(train_feat, test_feat):
    model = get_sklearn_model("LGB")    
    model.fit(get_predictors_df(train_feat), get_target_df(train_feat))    
    predicted = pd.DataFrame(model.predict_proba(get_predictors_df(test_feat))[:,1])

    model1 = get_sklearn_model("XGB")    
    model1.fit(get_predictors_df(train_feat), get_target_df(train_feat))    
    predicted1 = pd.DataFrame(model1.predict_proba(get_predictors_df(test_feat))[:,1])

    return 0.5*predicted+0.5*predicted1

In [156]:
predicted=classifier_df_simple(train_f3, test_f3,'LGB',params)

In [44]:
p=classifier_df_stack(train_f3, test_f3)

In [45]:
p.head()

,0
0,0.004812
1,0.141817
2,0.004435
3,0.009379
4,0.032814


In [157]:
predicted.head()

,0
0,0.005123
1,0.128401
2,0.004079
3,0.009573
4,0.031057


In [158]:
#生成结果数据
result=output_predicted(predicted,'sf3_LGB.csv',test_f3)

In [159]:
result.head()

,user_id,coupon_id,date_received,Probability
0,4129537,9983,20160712,0.005
1,6949378,3429,20160706,0.128
2,2166529,6928,20160727,0.004
3,2166529,1808,20160727,0.010
4,6172162,6500,20160708,0.031


In [160]:
result.to_csv('sf3_lgb.csv',header=False,index=False,sep=',')

: 